In [ ]:
from google.colab import drive
drive.mount('/content/drive')

PROJECT_DIR = '/content/drive/MyDrive/Programming/Robotics/so-101'
%cd $PROJECT_DIR

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Programming/Robotics/so-101


# 🤗 x 🦾: Training ACT with LeRobot Notebook

Welcome to the **LeRobot ACT training notebook**! This notebook provides a ready-to-run setup for training imitation learning policies using the [🤗 LeRobot](https://github.com/huggingface/lerobot) library.

In this example, we train an `ACT` policy using a dataset hosted on the [Hugging Face Hub](https://huggingface.co/), and optionally track training metrics with [Weights & Biases (wandb)](https://wandb.ai/).

## ⚙️ Requirements
- A Hugging Face dataset repo ID containing your training data (`--dataset.repo_id=YOUR_USERNAME/YOUR_DATASET`)
- Optional: A [wandb](https://wandb.ai/) account if you want to enable training visualization
- Recommended: GPU runtime (e.g., NVIDIA A100) for faster training

## ⏱️ Expected Training Time
Training with the `ACT` policy for 100,000 steps typically takes **about 1.5 hours on an NVIDIA A100** GPU. On less powerful GPUs or CPUs, training may take significantly longer.

## Example Output
Model checkpoints, logs, and training plots will be saved to the specified `--output_dir`. If `wandb` is enabled, progress will also be visualized in your wandb project dashboard.


## Install conda
This cell uses `condacolab` to bootstrap a full Conda environment inside Google Colab.


In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:11
🔁 Restarting kernel...


## Install LeRobot
This cell clones the `lerobot` repository from Hugging Face, installs FFmpeg (version 7.1.1), and installs the package in editable mode.


In [ ]:
!conda install ffmpeg=7.1.1 -c conda-forge
!cd lerobot && pip install -e .

Channels:
 - conda-forge
Platform: linux-64
Solving environment: \ | / - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.11.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

Obtaining file:///content/drive/MyDrive/Programming/Robotics/so-101/lerobot
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.9/28.

Install the datasets version expected by LeRobtot 0.1.0

In [ ]:
!pip uninstall -y datasets
!pip install datasets==3.6.0

Found existing installation: datasets 4.4.2
Uninstalling datasets-4.4.2:
  Successfully uninstalled datasets-4.4.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.10.0
    Uninstalling fsspec-2025.10.0:
      Successfully uninstalled fsspec-2025.10.0
  Attempting uninstall: dill
    Found existing installation: dill 0.4.0
    Uninstalling dill-0.4.0:
      Successfully uninstalled dill-0.4.0
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.18
    Uninstalling multiprocess-0.70.18:
      Successfully uninstalled multiprocess-0.70.18


## Weights & Biases login
This cell logs you into Weights & Biases (wandb) to enable experiment tracking and logging.

In [ ]:
!wandb login

wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: varsenii to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Download the dataset

In [ ]:
HF_USER = 'varsenii'
DATASET = "lego_pick_and_place"

!hf download $HF_USER/$DATASET --repo-type dataset

Fetching 158 files: 100% 158/158 [00:50<00:00,  3.10it/s]
Download complete: : 301MB [00:50, 3.43MB/s]              /root/.cache/huggingface/hub/datasets--varsenii--lego_pick_and_place/snapshots/7c8e1874ff88353bae78f39d18e8ffa5b92ae1f0
Download complete: : 301MB [00:50, 5.91MB/s]


In [ ]:
DATASET_PATH='/root/.cache/huggingface/hub/datasets--varsenii--lego_pick_and_place/snapshots/7c8e1874ff88353bae78f39d18e8ffa5b92ae1f0'

## Start training ACT with LeRobot

This cell runs the `train.py` script from the `lerobot` library to train a robot control policy.  

Make sure to adjust the following arguments to your setup:

1. `--dataset.repo_id=YOUR_HF_USERNAME/YOUR_DATASET`:  
   Replace this with the Hugging Face Hub repo ID where your dataset is stored, e.g., `pepijn223/il_gym0`.

2. `--policy.type=act`:  
   Specifies the policy configuration to use. `act` refers to [configuration_act.py](../lerobot/common/policies/act/configuration_act.py), which will automatically adapt to your dataset’s setup (e.g., number of motors and cameras).

3. `--output_dir=outputs/train/...`:  
   Directory where training logs and model checkpoints will be saved.

4. `--job_name=...`:  
   A name for this training job, used for logging and Weights & Biases.

5. `--policy.device=cuda`:  
   Use `cuda` if training on an NVIDIA GPU. Use `mps` for Apple Silicon, or `cpu` if no GPU is available.

6. `--wandb.enable=true`:  
   Enables Weights & Biases for visualizing training progress. You must be logged in via `wandb login` before running this.

In [ ]:
OUTPUT_DIR = '/content/drive/MyDrive/Programming/Robotics/so-101/output'

POLICY = "act"
TASK = POLICY + "_" + DATASET

# Training configuration
NUM_SAMPLES = 15585
BATCH_SIZE = 32
EPOCHS = 10
STEPS_PER_EPOCH = int(NUM_SAMPLES / BATCH_SIZE)
STEPS_TOTAL = STEPS_PER_EPOCH * EPOCHS
EVAL_FREQ = STEPS_PER_EPOCH // 100
LOG_FREQ = STEPS_PER_EPOCH // 100
SAVE_FREQ = STEPS_PER_EPOCH // 10

print(f'Policy: {POLICY}')
print(f'Steps per epoch: {STEPS_PER_EPOCH}')
print(f'Total steps: {STEPS_TOTAL}')
print(f'Eval frequency: {EVAL_FREQ}')
print(f'Log frequency: {LOG_FREQ}')
print(f'Save frequency: {SAVE_FREQ}')

Policy: act
Steps per epoch: 487
Total steps: 4870
Eval frequency: 4
Log frequency: 4
Save frequency: 48


In [ ]:
!cd lerobot && python lerobot/scripts/train.py \
  --dataset.root=$DATASET_PATH \
  --dataset.repo_id=$HF_USER/$DATASET \
  --policy.type=$POLICY \
  --policy.device=cuda \
  --job_name=$TASK \
  --output_dir=$OUTPUT_DIR/train/$TASK \
  --wandb.enable=true \
  --batch_size=$BATCH_SIZE \
  --steps=$STEPS_TOTAL \
  --log_freq=$LOG_FREQ \
  --eval_freq=$EVAL_FREQ \
  --save_freq=$SAVE_FREQ

INFO 2026-01-02 18:28:15 ts/train.py:111 {'batch_size': 32,
 'dataset': {'episodes': None,
             'image_transforms': {'enable': False,
                                  'max_num_transforms': 3,
                                  'random_order': False,
                                  'tfs': {'brightness': {'kwargs': {'brightness': [0.8,
                                                                                   1.2]},
                                                         'type': 'ColorJitter',
                                                         'weight': 1.0},
                                          'contrast': {'kwargs': {'contrast': [0.8,
                                                                               1.2]},
                                                       'type': 'ColorJitter',
                                                       'weight': 1.0},
                                          'hue': {'kwargs': {'hue': [-0.05,
                

## Login into Hugging Face Hub
Now after training is done login into the Hugging Face hub and upload the last checkpoint

In [ ]:
!huggingface-cli login

In [ ]:
!huggingface-cli upload ${HF_USER}/il_sim_test0 \
  /content/lerobot/outputs/train/il_sim_test0/checkpoints/last/pretrained_model